In [1]:
import calendar
import pandas as pd
import panel as pn
from datetime import date, timedelta
from sqlalchemy import create_engine
from itables import show
pn.extension('tabulator')

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

engine = create_engine("sqlite:///c:\\ruby\\portmy\\db\\development.sqlite3")
conmy = engine.connect()

engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"
one_path = "\\Users\\User\\OneDrive\\Documents\\Data\\"

today = date.today()
today

datetime.date(2022, 5, 15)

### Tables in the process

In [2]:
cols = 'trade name qty price active reason market xdate'.split()
colt = 'trans name spd reason qty target current percent active xdate'.split()

In [3]:
format_dict = {
    'qty':'{:,}','available_qty':'{:,}',
    'price':'{:.2f}','cost':'{:.2f}','max_price':'{:.2f}','min_price':'{:.2f}',
    'buy_target':'{:.2f}','sell_target':'{:.2f}','target':'{:.2f}','current':'{:.2f}',
    'beta':'{:.2f}',
    'amount':'{:,.2f}','sell_amt':'{:,.2f}','volume':'{:,.2f}',
    'percent':'{:,.2f}',
    'shares':'{:,}',    
    'q4':'{:.4f}','q3':'{:.4f}','q2':'{:.4f}','q1':'{:.4f}','dividend':'{:.4f}',
    'xdate':'{:%Y-%m-%d}','paiddate':'{:%Y-%m-%d}',
}

In [4]:
sql = '''
SELECT *
FROM orders
ORDER BY id DESC
LIMIT 1'''
orders = pd.read_sql(sql, conlite)
orders

,id,trade,name,qty,price,active,reason,market,xdate
0,21,S,MAKRO,1500,40.5,0,C40.40,SET,2022-03-03


In [5]:
names = orders['name']
name = names.to_string(index=False)

In [6]:
sql = '''
SELECT *
FROM dividend
WHERE name = '%s'
'''
sql = sql % name
dividend = pd.read_sql(sql, const)
dividend.drop(['PRICE', 'PERCENT'], axis=1, inplace=True)
dividend.columns = dividend.columns.str.lower()
dividend['shares'] = dividend['shares'].astype('int64')
dividend['xdate'] = pd.to_datetime(dividend['xdate'])
dividend['paiddate'] = pd.to_datetime(dividend['paiddate'])
dividend.style.format(format_dict)

,name,q4,q3,q2,q1,dividend,shares,xdate,paiddate,kind,actual
0,MAKRO,0.3200,0.0000,0.4000,0.0000,0.7200,"7,500",2022-03-03,2022-05-19,,0


In [7]:
sql = """
SELECT * FROM stocks WHERE name = '%s'
"""
sql = sql % name
print(sql)

stocks = pd.read_sql(sql, conlite)
stocks.style.format(format_dict)


SELECT * FROM stocks WHERE name = 'MAKRO'



,id,name,max_price,min_price,status,buy_target,sell_target,volume,beta,cost,qty,buy_spread,sell_spread,available_qty,bl,sh,reason,market
0,704,MAKRO,57.25,35.00,I,0.00,40.50,618.00,1.00,40.40,"1,500",-8,8,"7,500",0,0,C40.40,SET


### Create orders from stocks after adjust stocks in port_lite from consensus

In [8]:
sql = '''
DELETE FROM orders'''
rp = conlite.execute(sql)
rp.rowcount

21

In [9]:
sql = '''
SELECT status AS trade, name, qty, buy_target AS price, 0 AS active, reason, market
FROM stocks
WHERE status IN ('B','O')
ORDER BY name'''
df_buy = pd.read_sql(sql, conlite)
df_buy

,trade,name,qty,price,active,reason,market
0,O,AIT,0,0.0,0,XXX,SET
1,O,BBL,2400,124.0,0,1L,SET50
2,O,BCPG,0,0.0,0,XXX,SET100
3,O,BDMS,4200,26.0,0,1L,SET50
4,O,BGC,10000,9.8,0,DOS,SET
5,O,BH,0,0.0,0,XXX,SET50
6,O,BPP,0,0.0,0,XXX,SET100
7,O,COTTO,0,0.0,0,XXX,SET
8,B,DIF,10000,14.2,0,DOS,SET
9,O,ESSO,0,0.0,0,XXX,SET100


In [10]:
sql = '''
SELECT status AS trade, name, qty, sell_target AS price, 0 AS active, reason, market
FROM stocks
WHERE status IN ('I','S')'''
df_sell = pd.read_sql(sql, conlite)
df_sell

,trade,name,qty,price,active,reason,market
0,S,PTTGC,3000,58.00,0,CUT,SET50
1,S,STA,2400,29.50,0,CUT,SET100
2,I,RATCH,2000,38.50,0,CUT,SET50
3,I,NER,9000,7.50,0,CP1S,SET
4,S,SYNEX,3000,28.25,0,CP1S,SET100
5,I,BCH,3000,21.40,0,CP1S,SET100
6,I,DOHOME,2400,20.30,0,C20.25,SET100
7,I,PTT,3000,37.00,0,C38.50,SET50
8,I,IP,2500,20.30,0,CP1S,mai
9,I,MAKRO,1500,40.50,0,C40.40,SET


In [11]:
df_order = df_buy.append(df_sell, ignore_index=True)
df_order.shape

(33, 7)

In [12]:
df_order.groupby(['trade']).count()

,name,qty,price,active,reason,market
trade,,,,,,
B,4,4,4,4,4,4
I,7,7,7,7,7,7
O,19,19,19,19,19,19
S,3,3,3,3,3,3


In [13]:
df_order.loc[df_order.trade == 'I','trade'] = 'S'
df_order.groupby(['trade']).count()

,name,qty,price,active,reason,market
trade,,,,,,
B,4,4,4,4,4,4
O,19,19,19,19,19,19
S,10,10,10,10,10,10


In [14]:
sql = '''
SELECT name, xdate
FROM dividend
'''
dividend = pd.read_sql(sql, const)
dividend.columns = dividend.columns.str.lower()
dividend['xdate'] = pd.to_datetime(dividend['xdate'])
dividend.shape

(83, 2)

In [15]:
df_merge = pd.merge(df_order, dividend, on='name', how='outer', indicator=True)
df_merge.shape

(96, 9)

### If there is no dividend record, must create dividend record before proceed

In [16]:
no_div = df_merge[df_merge['_merge'] == 'left_only']
no_div

,trade,name,qty,price,active,reason,market,xdate,_merge
0,O,AIT,0.0,0.0,0.0,XXX,SET,NaT,left_only
2,O,BCPG,0.0,0.0,0.0,XXX,SET100,NaT,left_only
5,O,BH,0.0,0.0,0.0,XXX,SET50,NaT,left_only
6,O,BPP,0.0,0.0,0.0,XXX,SET100,NaT,left_only
7,O,COTTO,0.0,0.0,0.0,XXX,SET,NaT,left_only
9,O,ESSO,0.0,0.0,0.0,XXX,SET100,NaT,left_only
10,O,GGC,0.0,0.0,0.0,XXX,SET,NaT,left_only
16,O,RBF,0.0,0.0,0.0,XXX,SET100,NaT,left_only
17,O,SIS,0.0,0.0,0.0,XXX,SET,NaT,left_only
18,O,SMPC,0.0,0.0,0.0,XXX,SET,NaT,left_only


### Rerun from here

In [17]:
mask = df_merge['_merge'] == 'both'
df_orders = df_merge[mask].copy()
df_orders.drop(['_merge'], axis=1, inplace=True)
df_orders['qty'] = df_orders['qty'].astype('int64')
df_orders['active'] = df_orders['active'].astype('int64')
df_orders['xdate'] = df_orders['xdate'].astype('str')
df_orders.groupby('trade').count()

,name,qty,price,active,reason,market,xdate
trade,,,,,,,
B,4,4,4,4,4,4,4
O,6,6,6,6,6,6,6
S,10,10,10,10,10,10,10


### Set xdate 2 weeks window of first Tuesday and second Friday

In [18]:
fm_date = '2022-05-03'

### Restart and Run All Cells

In [19]:
fltr_sts_o = (df_orders.trade == 'O') 
df_sts_o = df_orders[fltr_sts_o]
df_sts_o

,trade,name,qty,price,active,reason,market,xdate
1,O,BBL,2400,124.0,0,1L,SET50,2022-04-22
3,O,BDMS,4200,26.0,0,1L,SET50,2022-03-09
4,O,BGC,10000,9.8,0,DOS,SET,2022-05-25
12,O,IVL,2400,45.0,0,XXX,SET50,2022-05-27
14,O,KBANK,1500,140.0,0,5pct,SET50,2022-04-18
15,O,KKP,3000,68.0,0,3L,SET100,2022-04-28


In [20]:
fltr_sts_b = (df_orders.trade == 'B') 
df_sts_b = df_orders[fltr_sts_b]
df_sts_b

,trade,name,qty,price,active,reason,market,xdate
8,B,DIF,10000,14.2,0,DOS,SET,2022-05-17
11,B,HREIT,10000,8.2,0,DOS,SET,2022-03-04
13,B,JASIF,10000,10.8,0,DOS,SET,2022-05-24
22,B,WHART,10000,10.5,0,RD10pct,SET,2022-06-01


In [21]:
fltr_sts_s = (df_orders.trade == 'S')
df_sts_s = df_orders[fltr_sts_s]
df_sts_s.sort_values('name',ascending=True)

,trade,name,qty,price,active,reason,market,xdate
28,S,BCH,3000,21.40,0,CP1S,SET100,2022-05-10
29,S,DOHOME,2400,20.30,0,C20.25,SET100,2022-02-02
31,S,IP,2500,20.30,0,CP1S,mai,2022-02-02
32,S,MAKRO,1500,40.50,0,C40.40,SET,2022-03-03
26,S,NER,9000,7.50,0,CP1S,SET,2022-04-19
30,S,PTT,3000,37.00,0,C38.50,SET50,2022-03-03
23,S,PTTGC,3000,58.00,0,CUT,SET50,2022-02-28
25,S,RATCH,2000,38.50,0,CUT,SET50,2022-03-17
24,S,STA,2400,29.50,0,CUT,SET100,2022-04-19
27,S,SYNEX,3000,28.25,0,CP1S,SET100,2022-03-10


In [22]:
frames = [df_sts_b, df_sts_o, df_sts_s]
df_orders = pd.concat(frames, sort=False)
df_orders.sort_values(['trade','name'],ascending=True)

,trade,name,qty,price,active,reason,market,xdate
8,B,DIF,10000,14.20,0,DOS,SET,2022-05-17
11,B,HREIT,10000,8.20,0,DOS,SET,2022-03-04
13,B,JASIF,10000,10.80,0,DOS,SET,2022-05-24
22,B,WHART,10000,10.50,0,RD10pct,SET,2022-06-01
1,O,BBL,2400,124.00,0,1L,SET50,2022-04-22
3,O,BDMS,4200,26.00,0,1L,SET50,2022-03-09
4,O,BGC,10000,9.80,0,DOS,SET,2022-05-25
12,O,IVL,2400,45.00,0,XXX,SET50,2022-05-27
14,O,KBANK,1500,140.00,0,5pct,SET50,2022-04-18
15,O,KKP,3000,68.00,0,3L,SET100,2022-04-28


In [23]:
df_orders.loc[df_orders.trade == 'O','trade'] = 'B'
df_orders.groupby(['trade']).count()

,name,qty,price,active,reason,market,xdate
trade,,,,,,,
B,10,10,10,10,10,10,10
S,10,10,10,10,10,10,10


In [24]:
file_name = 'orders.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

df_orders[cols].sort_values(['trade','name'],ascending=[True,True]).to_csv(output_file, header=True, index=False)
df_orders[cols].sort_values(['trade','name'],ascending=[True,True]).to_csv(data_file, header=True, index=False)
df_orders[cols].sort_values(['trade','name'],ascending=[True,True]).to_csv(box_file, header=True, index=False)
df_orders[cols].sort_values(['trade','name'],ascending=[True,True]).to_csv(one_file, header=True, index=False)

In [25]:
rcds = df_orders.values.tolist()
len(rcds)

20

In [26]:
for rcd in rcds:
    print(rcd)

['B', 'DIF', 10000, 14.2, 0, 'DOS', 'SET', '2022-05-17']
['B', 'HREIT', 10000, 8.2, 0, 'DOS', 'SET', '2022-03-04']
['B', 'JASIF', 10000, 10.8, 0, 'DOS', 'SET', '2022-05-24']
['B', 'WHART', 10000, 10.5, 0, 'RD10pct', 'SET', '2022-06-01']
['B', 'BBL', 2400, 124.0, 0, '1L', 'SET50', '2022-04-22']
['B', 'BDMS', 4200, 26.0, 0, '1L', 'SET50', '2022-03-09']
['B', 'BGC', 10000, 9.8, 0, 'DOS', 'SET', '2022-05-25']
['B', 'IVL', 2400, 45.0, 0, 'XXX', 'SET50', '2022-05-27']
['B', 'KBANK', 1500, 140.0, 0, '5pct', 'SET50', '2022-04-18']
['B', 'KKP', 3000, 68.0, 0, '3L', 'SET100', '2022-04-28']
['S', 'PTTGC', 3000, 58.0, 0, 'CUT', 'SET50', '2022-02-28']
['S', 'STA', 2400, 29.5, 0, 'CUT', 'SET100', '2022-04-19']
['S', 'RATCH', 2000, 38.5, 0, 'CUT', 'SET50', '2022-03-17']
['S', 'NER', 9000, 7.5, 0, 'CP1S', 'SET', '2022-04-19']
['S', 'SYNEX', 3000, 28.25, 0, 'CP1S', 'SET100', '2022-03-10']
['S', 'BCH', 3000, 21.4, 0, 'CP1S', 'SET100', '2022-05-10']
['S', 'DOHOME', 2400, 20.3, 0, 'C20.25', 'SET100', '202

In [27]:
sql = """
INSERT INTO orders (trade, name, qty, price, active,
reason, market, xdate)
VALUES (?, ?, ?, ?, ?, ?, ?, ?)"""
print(sql)


INSERT INTO orders (trade, name, qty, price, active,
reason, market, xdate)
VALUES (?, ?, ?, ?, ?, ?, ?, ?)


In [28]:
for rcd in rcds:
    conlite.execute(sql, rcd)

### After call Ord-log.bat

In [29]:
def categorise(row):  
    if row['trade'] == 'B':
        return 'Buy'
    else:
        return 'Sell'

In [30]:
df = pd.read_csv('../data/orders-log.csv')
df['trans'] = df.apply(lambda row: categorise(row), axis=1)

df_tab = pn.widgets.Tabulator(df[colt], layout='fit_data', width=800)
df_tab

Tabulator(layout='fit_data', value=   trans    name  ..., width=800)

In [31]:
df[colt]

,trans,name,spd,reason,qty,target,current,percent,active,xdate
0,Buy,BBL,-2,1L,2400,124.00,125.00,-0.79%,0,2022-04-22
1,Buy,BDMS,-4,1L,4200,26.00,27.00,+3.85%,0,2022-03-09
2,Buy,BGC,-4,DOS,10000,9.80,10.00,+2.04%,0,2022-05-25
3,Buy,DIF,-1,DOS,10000,14.20,14.30,0.00%,0,2022-05-17
4,Buy,HREIT,-3,DOS,10000,8.20,8.35,+0.60%,0,2022-03-04
5,Buy,IVL,-6,XXX,2400,45.00,46.50,+5.08%,0,2022-05-27
6,Buy,JASIF,-2,DOS,10000,10.80,11.00,+0.92%,0,2022-05-24
7,Buy,KBANK,-4,5pct,1500,140.00,142.00,-0.35%,0,2022-04-18
8,Buy,KKP,-5,3L,3000,68.00,69.25,-1.42%,0,2022-04-28
9,Buy,MST,-6,XXX,10000,11.00,11.60,-0.85%,0,2022-03-10
